In [26]:
# paibox 和 spikingjelly 的推理结果比较，输出为 csv 文件
# 需要修改数据集路径DVS128GESTURE_DATA_DIR
import torch
import numpy as np
import paibox as pb
import csv

from spikingjelly.activation_based import neuron, functional, surrogate, layer
from spikingjelly.datasets.dvs128_gesture import DVS128Gesture
from torch.utils.data import DataLoader

from infer_conv2int import PythonNet
from voting import voting
vthr_list = [53,146,134,118,124,97,151,76,113,99,121,90] # vthr from model_parameters_conv2int.py

SIM_TIMESTEP = 4 # <=16
DVS128GESTURE_DATA_DIR = '../15-classify_dvsg/DVS128Gesture'

# Dataloader
# test_set = DVS128Gesture(DVS128GESTURE_DATA_DIR, train=False, data_type='frame', split_by='number', duration=131072)
# test_data_loader = DataLoader(
#     dataset=test_set,
#     batch_size=1,
#     shuffle=False,
#     num_workers=1,
#     drop_last=False,
#     pin_memory=True)

# PAIBox网络定义
class Conv2d_Net(pb.Network):
    def __init__(self, channels):
        super().__init__()

        self.i0 = pb.InputProj(input=None, shape_out=(2, 64, 64))
        # self.n0 = pb.LIF((channels*1, 64, 64), reset_v=0, tick_wait_start=1, threshold=1) # conv3x3
        # self.conv2d_0 = pb.Conv2d(self.i0, self.n0, kernel=np.zeros((channels*1, 2, 5, 5)).astype(np.int8), padding=2, stride=1)

        self.n1 = pb.LIF((channels*2, 32, 32), reset_v=0, tick_wait_start=1, threshold=1) # convpool3x3
        self.conv2d_1 = pb.Conv2d(self.i0, self.n1, kernel=np.zeros((channels*2, 2, 5, 5)).astype(np.int8), padding=2, stride=2)

        self.n2 = pb.LIF((channels*2, 32, 32), reset_v=0, tick_wait_start=2, threshold=1) # conv3x3
        self.conv2d_2 = pb.Conv2d(self.n1, self.n2, kernel=np.zeros((channels*2, channels*2, 5, 5)).astype(np.int8), padding=2, stride=1)

        self.n3 = pb.LIF((channels*4, 16, 16), reset_v=0, tick_wait_start=3, threshold=1) # convpool3x3
        self.conv2d_3 = pb.Conv2d(self.n2, self.n3, kernel=np.zeros((channels*4, channels*2, 5, 5)).astype(np.int8), padding=2, stride=2)

        self.n4 = pb.LIF((channels*4, 16, 16), reset_v=0, tick_wait_start=4, threshold=1) # conv3x3
        self.conv2d_4 = pb.Conv2d(self.n3, self.n4, kernel=np.zeros((channels*4, channels*4, 5, 5)).astype(np.int8), padding=2, stride=1)

        self.n5 = pb.LIF((channels*8, 8, 8), reset_v=0, tick_wait_start=5, threshold=1) # convpool3x3
        self.conv2d_5 = pb.Conv2d(self.n4, self.n5, kernel=np.zeros((channels*8, channels*4, 5, 5)).astype(np.int8), padding=2, stride=2)

        self.n6 = pb.LIF((channels*8, 8, 8), reset_v=0, tick_wait_start=6, threshold=1) # conv3x3
        self.conv2d_6 = pb.Conv2d(self.n5, self.n6, kernel=np.zeros((channels*8, channels*8, 5, 5)).astype(np.int8), padding=2, stride=1)

        self.n10 = pb.LIF(channels*8 * 4 * 4, reset_v=0, tick_wait_start=7, threshold=1) # fc
        self.fc_0 = pb.FullConn(self.n6, self.n10, conn_type=pb.SynConnType.All2All)

        self.n11 = pb.LIF(channels*8 * 4 * 4, reset_v=0, tick_wait_start=8, threshold=1) # fc
        self.fc_1 = pb.FullConn(self.n10, self.n11, conn_type=pb.SynConnType.All2All)

        self.n12 = pb.LIF(channels*8 * 4 * 4, reset_v=0, tick_wait_start=9, threshold=1) # fc
        self.fc_1 = pb.FullConn(self.n11, self.n12, conn_type=pb.SynConnType.All2All)

        self.n13 = pb.LIF(110, reset_v=0, tick_wait_start=10, threshold=1) # fc
        self.fc_3 = pb.FullConn(self.n12, self.n13, conn_type=pb.SynConnType.All2All)

        self.probe1 = pb.Probe(self.n13, "spike")

# PAIBox仿真器
pb_net = Conv2d_Net(4)
sim = pb.Simulator(pb_net)


# test(test_num=50)
pb.BACKEND_CONFIG.target_chip_addr = (0, 0)

# print(pb_net.conv2d_0.name)
# print(type(pb_net.n0))
# print(pb_net.n0.shape_in)
# print(pb_net.n0.shape_out)
# print(type(pb_net.n0[0,:,:]))
# print(pb_net.n0[0,:,:].shape_in)
# print(pb_net.n0[0,:,:].shape_out)
# n1_0_1 = (pb_net.n11,pb_net.n11)
# print(type(n1_0_1))

In [27]:
mapper = pb.Mapper()

mapper.build(pb_net)

In [28]:
graph_info = mapper.compile(
    weight_bit_optimization=True, grouping_optim_target="both"
)

In [29]:
# #N of cores required
print("Core required:", graph_info["n_core_required"])

mapper.export(
    write_to_file=True, fp="./debug", format="npy", export_core_params=False
)

# Clear all the results
mapper.clear()

Core required: 381
